# Python OOPS Questions

---



### Q: What is Object-Oriented Programming (OOP)

**Answer:**  
Object-Oriented Programming (OOP) is a programming paradigm that organizes software design around **objects**—bundles of *state* (data/attributes) and *behavior* (methods).  
Core goals: **modularity**, **reusability**, **extensibility**, and **maintainability**.  
Key principles (often remembered as *A PIE*): **Abstraction**, **Polymorphism**, **Inheritance**, **Encapsulation**.

**Why it matters:** OOP maps well to real-world domains, reduces code duplication via reuse/extension, and encourages clean interfaces.


In [ ]:

# Tiny demo: objects bundle state + behavior
class Counter:
    def __init__(self, start=0):
        self.value = start  # state
    def inc(self):         # behavior
        self.value += 1
    def __repr__(self):
        return f"Counter(value={self.value})"

c = Counter()
c.inc(); c.inc()
c


### Q: What is a class in OOP

**Answer:**  
A **class** is a blueprint for creating objects. It defines the **attributes** (data) and **methods** (behaviors) shared by all its instances. Classes enable code organization, reuse, and type creation.


In [ ]:

class Dog:
    species = "Canis familiaris"  # class attribute (shared)
    def __init__(self, name):
        self.name = name           # instance attribute (per-object)
    def bark(self):
        return f"{self.name} says: Woof!"
Dog("Buddy").bark()


### Q: What is an object in OOP

**Answer:**  
An **object** (or instance) is a concrete realization of a class—i.e., data stored in memory with methods bound to it. Each object has its own state but shares behavior defined by the class.


In [ ]:

d1 = Dog("Milo")
d2 = Dog("Luna")
(d1.name, d2.name, d1 is d2)


### Q: What is the difference between abstraction and encapsulation

**Answer:**  
- **Abstraction**: *Expose only what’s necessary*—provide simple interfaces while hiding complex internals (focus on **what**).  
- **Encapsulation**: *Package data with the methods that operate on it* and **restrict direct access** to the internals (focus on **how to protect**).

Abstraction is about designing clean APIs; encapsulation is about data protection and boundaries.


In [ ]:

class Temperature:
    # Encapsulation: keep _celsius "internal"
    def __init__(self, celsius):
        self._celsius = float(celsius)
    # Abstraction: simple property interface
    @property
    def celsius(self):
        return self._celsius
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Below absolute zero")
        self._celsius = float(value)
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

t = Temperature(25)
(t.celsius, round(t.fahrenheit, 2))


### Q: What are dunder methods in Python

**Answer:**  
**Dunder methods** (double-underscore methods) customize built‑in behavior (construction, printing, arithmetic, iteration, context management, etc.). Examples: `__init__`, `__repr__`, `__str__`, `__len__`, `__add__`, `__iter__`, `__enter__`, `__exit__`, `__call__`.

They let objects integrate naturally with Python syntax and protocols.


In [ ]:

class Vector2D:
    def __init__(self, x, y):
        self.x, self.y = x, y
    def __add__(self, other):
        return Vector2D(self.x + other.x, self.y + other.y)
    def __repr__(self):
        return f"Vector2D({self.x}, {self.y})"

Vector2D(1, 2) + Vector2D(3, 4)


### Q: Explain the concept of inheritance in OOP

**Answer:**  
**Inheritance** lets a class (**child/subclass**) reuse and extend another class (**parent/superclass**). It promotes code reuse and polymorphism.

Types in Python: single, multiple, multilevel, hierarchical.


In [ ]:

class Animal:
    def speak(self): return "Some sound"

class Dog(Animal):
    def speak(self): return "Bark!"  # override

(Animal().speak(), Dog().speak())


### Q: What is polymorphism in OOP

**Answer:**  
**Polymorphism** means *same interface, different implementations*. Functions can work with objects of different types as long as they support the expected behavior (Pythonic **duck typing**).


In [ ]:

def make_it_speak(creature):
    # Works for any object that has .speak()
    return creature.speak()

class Cat:
    def speak(self): return "Meow"
make_it_speak(Dog()) , make_it_speak(Cat())


### Q: How is encapsulation achieved in Python

**Answer:**  
Python uses conventions and name mangling to discourage direct access:
- Prefix with `_name` (protected-by-convention).
- Use `__name` (name-mangled) for stronger avoidance.
- Provide **properties**/methods to control access, validation, and invariants.


In [ ]:

class BankAccount:
    def __init__(self, balance=0):
        self.__balance = balance  # name-mangled
    def deposit(self, amount):
        if amount <= 0: raise ValueError("amount must be +")
        self.__balance += amount
    def withdraw(self, amount):
        if amount <= 0 or amount > self.__balance:
            raise ValueError("invalid withdraw")
        self.__balance -= amount
    @property
    def balance(self):
        return self.__balance

acct = BankAccount(100); acct.deposit(50); acct.withdraw(20); acct.balance


### Q: What is a constructor in Python

**Answer:**  
A **constructor** initializes a new object’s state. In Python, the initializer is `__init__(self, ...)`, called after memory is allocated by `__new__`. Use it to set attributes and enforce invariants.


In [ ]:

class User:
    def __init__(self, username, email):
        if "@" not in email:
            raise ValueError("invalid email")
        self.username = username
        self.email = email

User("alice", "alice@example.com").__dict__


### Q: What are class and static methods in Python

**Answer:**  
- `@classmethod`: receives the class as first arg (`cls`). Used for **alternative constructors**, **class-wide state**, or behaviors tied to the type.
- `@staticmethod`: no auto `self/cls`. A namespaced utility function residing on the class for organization.


In [ ]:

class Date:
    def __init__(self, y, m, d): self.y, self.m, self.d = y, m, d
    @classmethod
    def from_string(cls, s):
        y, m, d = map(int, s.split("-")); return cls(y, m, d)
    @staticmethod
    def is_leap(year):
        return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

Date.is_leap(2000), Date.from_string("2024-02-29").__dict__


### Q: What is method overloading in Python

**Answer:**  
Classic **compile-time overloading** (same name, different signatures) isn’t native in Python. Instead, we use:
- **Default args/`*args`/`**kwargs`** to accept variants,
- **`functools.singledispatch`** for type-based generic functions,
- or conditional logic inside the method.


In [ ]:

from functools import singledispatch

@singledispatch
def area(shape):
    raise TypeError("unsupported type")
@area.register
def _(r: float):
    # treat float as radius of a circle
    import math; return math.pi * r * r
@area.register
def _(s: tuple):
    # treat tuple as rectangle (w, h)
    w, h = s; return w * h

area(2.0), area((3,4))


### Q: What is method overriding in OOP

**Answer:**  
**Overriding** occurs when a subclass provides a new implementation of a method defined in its superclass. Calls on subclass instances resolve to the overridden method via **dynamic dispatch**.


In [ ]:

class Parent:
    def greet(self): return "Hello from Parent"
class Child(Parent):
    def greet(self): return "Hello from Child"

Parent().greet(), Child().greet()


### Q: What is a property decorator in Python

**Answer:**  
`@property` turns a method into a **managed attribute**. It enables controlled access with optional validation via a setter and computed read-only or read‑write attributes, without changing the attribute access syntax.


In [ ]:

class Person:
    def __init__(self, age): self._age = age
    @property
    def age(self): return self._age
    @age.setter
    def age(self, value):
        if value < 0: raise ValueError("age must be >= 0")
        self._age = value

p = Person(20); p.age = 21; p.age


### Q: Why is polymorphism important in OOP

**Answer:**  
Polymorphism decouples *what* you do from *how* it’s done, enabling **flexible, extensible, testable** designs. New types can plug into existing code as long as they honor the expected interface (Liskov Substitution Principle).


### Q: What is an abstract class in Python

**Answer:**  
An **abstract class** defines an interface and possibly shared behavior, but cannot be instantiated. Use `abc.ABC` and `@abstractmethod` to force subclasses to implement required methods.


In [ ]:

from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def area(self): ...
class Square(Shape):
    def __init__(self, side): self.side = side
    def area(self): return self.side * self.side

Square(3).area()


### Q: What are the advantages of OOP

**Answer:**  
- **Modularity** and separation of concerns  
- **Reusability** via inheritance/composition  
- **Extensibility** through polymorphism  
- **Maintainability** with encapsulation and clear interfaces  
- **Testability** with small, coherent units


### Q: What is the difference between a class variable and an instance variable

**Answer:**  
- **Class variable**: shared by all instances (defined at class level).  
- **Instance variable**: unique per object (usually set in `__init__`).

Be careful not to mutate shared mutable class variables unexpectedly.


In [ ]:

class Bag:
    label = "GENERIC"        # class variable
    def __init__(self, size):
        self.size = size     # instance variable

b1, b2 = Bag(10), Bag(20)
(b1.label, b2.label, b1.size, b2.size)


### Q: What is multiple inheritance in Python

**Answer:**  
**Multiple inheritance** lets a class inherit from multiple bases. Python uses **MRO (C3 linearization)** to resolve method lookup order.

Prefer **mixins** and keep hierarchies simple to avoid the *diamond problem* pitfalls.


In [ ]:

class Flyer:
    def move(self): return "flying"
class Swimmer:
    def move(self): return "swimming"
class Duck(Flyer, Swimmer):
    pass

Duck().move(), Duck.__mro__


### Q: Explain the purpose of ‘__str__’ and ‘__repr__’ methods in Python

**Answer:**  
- `__repr__`: unambiguous, often a *developer-facing* representation; aim to be valid Python or detailed. Used by `repr()` and interactive consoles.
- `__str__`: readable, *user-facing* string. Used by `print()`.
If only `__repr__` is defined, Python may fall back to it for `str()`.


In [ ]:

class Item:
    def __init__(self, name, price):
        self.name, self.price = name, price
    def __repr__(self):
        return f"Item(name={self.name!r}, price={self.price!r})"
    def __str__(self):
        return f"{self.name} - ${self.price:.2f}"

repr(Item("Pen", 1.5)), str(Item("Pen", 1.5))


### Q: What is the significance of the ‘super()’ function in Python

**Answer:**  
`super()` returns a proxy to delegate method calls to the next class in the **MRO**. It ensures correct cooperative multiple inheritance and avoids hard-coding parent names.


In [ ]:

class A:
    def greet(self): return "A"
class B(A):
    def greet(self):
        return super().greet() + " -> B"
class C(B):
    def greet(self):
        return super().greet() + " -> C"

C().greet()


### Q: What is the significance of the __del__ method in Python

**Answer:**  
`__del__` is a **finalizer** called when an object is about to be garbage-collected. Use sparingly: timing isn’t guaranteed, exceptions are ignored, and cycles can prevent calls. Prefer **context managers** (`with`) and `__enter__/__exit__` for resource management.


In [ ]:

class FileHolder:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "w")
    def __del__(self):
        try:
            self.f.close()
        except Exception:
            pass

# Better: use context managers instead of relying on __del__


### Q: What is the difference between @staticmethod and @classmethod in Python

**Answer:**  
- `@classmethod(cls, ...)` gets the **class**; good for alt constructors and class-level behavior.
- `@staticmethod(...)` gets **no implicit first arg**; good for utility helpers colocated with the class.


In [ ]:

class MathOps:
    factor = 10
    @classmethod
    def scaled_sum(cls, a, b):
        return cls.factor * (a + b)
    @staticmethod
    def diff(a, b):
        return a - b

MathOps.scaled_sum(2,3), MathOps.diff(9,4)


### Q: How does polymorphism work in Python with inheritance

**Answer:**  
Through **dynamic dispatch**: method calls are resolved at runtime based on the object’s actual type. Subclasses override methods; base-type references can point to subclass objects, and the overridden method is invoked.


In [ ]:

class Bird:
    def fly(self): return "Bird flying"
class Penguin(Bird):
    def fly(self): return "Penguins don't fly; they waddle!"

def takeoff(bird: Bird):
    return bird.fly()

takeoff(Bird()), takeoff(Penguin())


### Q: What is method chaining in Python OOP

**Answer:**  
**Method chaining** returns `self` (or another object) from methods so calls can be **chained** in a single expression. Improves fluency for builder/DSL‑style APIs.


In [ ]:

class Query:
    def __init__(self):
        self.filters = []
    def where(self, cond):
        self.filters.append(cond); return self
    def limit(self, n):
        self.n = n; return self
    def build(self):
        return {"filters": self.filters, "limit": getattr(self, "n", None)}

Query().where("age>18").where("country='IN'").limit(10).build()


### Q: What is the purpose of the __call__ method in Python?

**Answer:**  
`__call__` makes an object **callable like a function**. Useful for function objects, configurable processors, and stateful strategies.


In [ ]:

class Multiplier:
    def __init__(self, k): self.k = k
    def __call__(self, x): return self.k * x

times3 = Multiplier(3)
times3(7)


# Practical Questions

---



### Q: Create a parent class Animal with a method speak() that prints a generic message. Create a child class Dog that overrides the speak() method to print 'Bark!'.

In [ ]:

class Animal:
    def speak(self):
        print("Some generic animal sound")

class Dog(Animal):
    def speak(self):
        print("Bark!")

Animal().speak()
Dog().speak()


### Q: Write a program to create an abstract class Shape with a method area(). Derive classes Circle and Rectangle from it and implement the area() method in both.

In [ ]:

from abc import ABC, abstractmethod
import math

class Shape(ABC):
    @abstractmethod
    def area(self):
        pass

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    def area(self):
        return math.pi * self.radius ** 2

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width, self.height = width, height
    def area(self):
        return self.width * self.height

Circle(3).area(), Rectangle(4, 5).area()


### Q: Implement a multi-level inheritance scenario where a class Vehicle has an attribute type. Derive a class Car and further derive a class ElectricCar that adds a battery attribute.

In [ ]:

class Vehicle:
    def __init__(self, type_):
        self.type = type_

class Car(Vehicle):
    def __init__(self, brand):
        super().__init__(type_="Car")
        self.brand = brand

class ElectricCar(Car):
    def __init__(self, brand, battery_kwh):
        super().__init__(brand)
        self.battery_kwh = battery_kwh

ec = ElectricCar("Tesla", 75)
(ec.type, ec.brand, ec.battery_kwh)


### Q: Demonstrate polymorphism by creating a base class Bird with a method fly(). Create two derived classes Sparrow and Penguin that override the fly() method.

In [ ]:

class Bird:
    def fly(self):
        return "Bird flying..."

class Sparrow(Bird):
    def fly(self):
        return "Sparrow flies swiftly!"

class Penguin(Bird):
    def fly(self):
        return "Penguin cannot fly; it swims!"

[cls().fly() for cls in (Bird, Sparrow, Penguin)]


### Q: Write a program to demonstrate encapsulation by creating a class BankAccount with private attributes balance and methods to deposit, withdraw, and check balance.

In [ ]:

class BankAccount:
    def __init__(self, balance=0):
        self.__balance = balance
    def deposit(self, amount):
        if amount <= 0: raise ValueError("amount must be positive")
        self.__balance += amount
    def withdraw(self, amount):
        if amount <= 0 or amount > self.__balance:
            raise ValueError("invalid withdraw")
        self.__balance -= amount
    def get_balance(self):
        return self.__balance

acct = BankAccount(1000)
acct.deposit(250)
acct.withdraw(300)
acct.get_balance()


### Q: Demonstrate runtime polymorphism using a method play() in a base class Instrument. Derive classes Guitar and Piano that implement their own version of play().

In [ ]:

class Instrument:
    def play(self):
        return "Instrument playing..."

class Guitar(Instrument):
    def play(self):
        return "Strumming guitar chords"

class Piano(Instrument):
    def play(self):
        return "Playing piano melody"

def concert(instrument: Instrument):
    return instrument.play()

concert(Guitar()), concert(Piano())


### Q: Create a class MathOperations with a class method add_numbers() to add two numbers and a static method subtract_numbers() to subtract two numbers.

In [ ]:

class MathOperations:
    @classmethod
    def add_numbers(cls, a, b):
        return a + b
    @staticmethod
    def subtract_numbers(a, b):
        return a - b

MathOperations.add_numbers(10, 7), MathOperations.subtract_numbers(10, 7)


### Q: Implement a class Person with a class method to count the total number of persons created.

In [ ]:

class Person:
    _count = 0
    def __init__(self, name):
        self.name = name
        Person._count += 1
    @classmethod
    def count(cls):
        return cls._count

p1, p2, p3 = Person("A"), Person("B"), Person("C")
Person.count()


### Q: Write a class Fraction with attributes numerator and denominator. Override the str method to display the fraction as 'numerator/denominator'.

In [ ]:

from math import gcd

class Fraction:
    def __init__(self, numerator, denominator):
        if denominator == 0:
            raise ZeroDivisionError("denominator cannot be zero")
        g = gcd(numerator, denominator)
        self.numerator = numerator // g
        self.denominator = denominator // g
    def __str__(self):
        return f"{self.numerator}/{self.denominator}"

print(str(Fraction(6, 8)))


### Q: Demonstrate operator overloading by creating a class Vector and overriding the add method to add two vectors.

In [ ]:

class Vector:
    def __init__(self, *components):
        self.components = tuple(components)
    def __add__(self, other):
        if len(self.components) != len(other.components):
            raise ValueError("dimension mismatch")
        return Vector(*[a+b for a,b in zip(self.components, other.components)])
    def __repr__(self):
        return f"Vector{self.components}"

Vector(1,2,3) + Vector(4,5,6)


### Q: Create a class Person with attributes name and age. Add a method greet() that prints 'Hello, my name is {name} and I am {age} years old.'

In [ ]:

class Person:
    def __init__(self, name, age):
        self.name, self.age = name, age
    def greet(self):
        print(f"Hello, my name is {self.name} and I am {self.age} years old.")

Person("Ishita", 29).greet()


### Q: Implement a class Student with attributes name and grades. Create a method average_grade() to compute the average of the grades.

In [ ]:

class Student:
    def __init__(self, name, grades):
        self.name = name
        self.grades = list(grades)
    def average_grade(self):
        return sum(self.grades)/len(self.grades) if self.grades else 0.0

Student("Vikram", [85, 90, 78]).average_grade()


### Q: Create a class Rectangle with methods set_dimensions() to set the dimensions and area() to calculate the area.

In [ ]:

class Rectangle:
    def __init__(self, width=0, height=0):
        self.width, self.height = width, height
    def set_dimensions(self, width, height):
        self.width, self.height = width, height
    def area(self):
        return self.width * self.height

r = Rectangle(); r.set_dimensions(7, 3); r.area()


### Q: Create a class Employee with a method calculate_salary() that computes the salary based on hours worked and hourly rate. Create a derived class Manager that adds a bonus to the salary.

In [ ]:

class Employee:
    def calculate_salary(self, hours_worked, hourly_rate):
        return hours_worked * hourly_rate

class Manager(Employee):
    def calculate_salary(self, hours_worked, hourly_rate, bonus=0):
        base = super().calculate_salary(hours_worked, hourly_rate)
        return base + bonus

Employee().calculate_salary(160, 25), Manager().calculate_salary(160, 25, bonus=2000)


### Q: Create a class Product with attributes name, price, and quantity. Implement a method total_price() that calculates the total price of the product.

In [ ]:

class Product:
    def __init__(self, name, price, quantity):
        self.name, self.price, self.quantity = name, float(price), int(quantity)
    def total_price(self):
        return self.price * self.quantity

Product("USB Cable", 199.0, 3).total_price()


### Q: Create a class Animal with an abstract method sound(). Create two derived classes Cow and Sheep that implement the sound() method.

In [ ]:

from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def sound(self): ...
class Cow(Animal):
    def sound(self): return "Moo"
class Sheep(Animal):
    def sound(self): return "Baa"

Cow().sound(), Sheep().sound()


### Q: Create a class Book with attributes title, author, and year_published. Add a method get_book_info() that returns a formatted string with the book's details.

In [ ]:

class Book:
    def __init__(self, title, author, year_published):
        self.title, self.author, self.year_published = title, author, int(year_published)
    def get_book_info(self):
        return f"'{self.title}' by {self.author} ({self.year_published})"

Book("Clean Code", "Robert C. Martin", 2008).get_book_info()


### Q: Create a class House with attributes address and price. Create a derived class Mansion that adds an attribute number_of_rooms.

In [ ]:

class House:
    def __init__(self, address, price):
        self.address, self.price = address, float(price)
class Mansion(House):
    def __init__(self, address, price, number_of_rooms):
        super().__init__(address, price)
        self.number_of_rooms = int(number_of_rooms)

m = Mansion("123 Palm Drive", 5_000_000, 12)
(m.address, m.price, m.number_of_rooms)
